In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np

# Making a single hidden layer neural network for `SkipGram`

In [ ]:
class DeepWalk(torch.nn.module):
        def __init__(self, adj_mat : 'list[list[int]]', feature_dimension : 'int'):
                super(DeepWalk, self).__init__()
                
                # Get the number of neurons in the input and output layer
                input_layer_size = int(len(adj_mat))
                output_layer_size = int(len(adj_mat))

                # Get the number of neurons in the hidden layer
                hidden_layer_size = feature_dimension

                self.linear_one = torch.nn.Linear(input_layer_size, hidden_layer_size)
                self.linear_two = torch.nn.Linear(hidden_layer_size, output_layer_size)

        def forward(self, x):
                self.layer_in = self.linear_one(x)
                # No activation
                self.layer_out = self.linear_two(self.layer_in)
                
                # 